In [7]:
!file /content/KoBart_trained_transfer.zip

/content/KoBart_trained_transfer.zip: Zip archive data, at least v2.0 to extract, compression method=deflate


In [8]:
!unzip /content/KoBart_trained_transfer.zip -d /content/KoBart_trained_transfer

Archive:  /content/KoBart_trained_transfer.zip
  inflating: /content/KoBart_trained_transfer/model.safetensors  
  inflating: /content/KoBart_trained_transfer/tokenizer_config.json  
  inflating: /content/KoBart_trained_transfer/special_tokens_map.json  
  inflating: /content/KoBart_trained_transfer/tokenizer.json  
  inflating: /content/KoBart_trained_transfer/generation_config.json  
  inflating: /content/KoBart_trained_transfer/config.json  


In [9]:
import os
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
import pandas as pd

# 모델과 토크나이저 로드
model_name = '/content/KoBart_trained_transfer'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to('cuda')

# 데이터셋 로드
data_path = '/content/processed_smilestyle_dataset.csv'
df = pd.read_csv(data_path)

# 스타일 목록 추출
styles = df['style'].unique()
print("Available styles:", styles)

# 스타일에 맞춰 문장 생성 함수
def change_style(input_text, style):
    input_text = f"<{style}> {input_text}"
    inputs = tokenizer(input_text, return_tensors='pt', max_length=128, truncation=True, padding='max_length').to('cuda')
    outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=5, early_stopping=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Available styles: ['gentle' 'sosim' 'informal' 'chat']


In [10]:
# 테스트할 문장
test_sentence = "안녕 오늘따라 날씨도 좋은데 오늘 뭐 할 계획이야?"

# 각 스타일별로 문장 생성 및 출력
for style in styles:
    styled_sentence = change_style(test_sentence, style)
    print(f"Style: {style}\nGenerated Text: {styled_sentence}\n")


Style: gentle
Generated Text: 안녕하십니까. 오늘 날씨도 좋네요. 오늘 뭐하실 계획이십니까?

Style: sosim
Generated Text: 안녕 오늘 오늘따라 날씨도 좋은데 오늘 뭐할 계획이야 혹시..?

Style: informal
Generated Text: 안녕 오늘따라 날씨도 좋은데 오늘 뭐 할 계획이야?

Style: chat
Generated Text: ᄒᄋ 오늘 날씨도 좋긴한데 오늘 뭐 할 생각?

